<a href="https://colab.research.google.com/github/Meitiann/INF2008-ML-Labs/blob/main/Project_Datapreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
##Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [175]:
train_bodies = pd.read_csv('train_bodies_processed.csv')
train_stances = pd.read_csv('train_stances_processed.csv')

In [176]:
train_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [177]:
train_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


Check Missing Values

In [178]:
train_bodies.isnull().sum()

,0
Body ID,0
articleBody,0


In [179]:
train_stances.isnull().sum()

,0
Headline,0
Body ID,0
Stance,0


Check and Remove Duplicates

In [180]:
train_stances_duplicates = train_stances.duplicated(subset=['Headline', 'Body ID', 'Stance'], keep='first').sum()
print(f"Total duplicate rows: {train_stances_duplicates}")


Total duplicate rows: 402


In [181]:
train_bodies_duplicates = train_bodies.duplicated(subset=['Body ID', 'articleBody'], keep='first').sum()
print(f"Total duplicate rows: {train_bodies_duplicates}")


Total duplicate rows: 0


In [182]:
train_stances_cleaned = train_stances.drop_duplicates(subset=['Headline', 'Body ID', 'Stance'], keep='first')


In [183]:
train_bodies_cleaned = train_bodies.drop_duplicates(subset=['Body ID', 'articleBody'], keep='first')

In [184]:
def preprocess_text(text):
  #remove extra whitespace

  text = ' '.join(text.split())

  return text

In [185]:
train_bodies_cleaned['articleBody'] = train_bodies_cleaned['articleBody'].apply(preprocess_text)

In [186]:
train_stances_cleaned['Headline'] = train_stances_cleaned['Headline'].apply(preprocess_text)

<ipython-input-186-210f99c88c29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_stances_cleaned['Headline'] = train_stances_cleaned['Headline'].apply(preprocess_text)


In [187]:
#Encode the stance labels
label_encoder = LabelEncoder()
train_stances_cleaned['Stance'] = label_encoder.fit_transform(train_stances_cleaned['Stance'])

<ipython-input-187-de0c3a57ded1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_stances_cleaned['Stance'] = label_encoder.fit_transform(train_stances_cleaned['Stance'])


In [188]:
stance_classes = label_encoder.classes_
print("Stance Classes:", stance_classes)

Stance Classes: ['agree' 'disagree' 'discuss' 'unrelated']


In [189]:
#Remove 'unrelated' and 'discuss' stances
train_stances_filtered = train_stances_cleaned[(train_stances_cleaned['Stance'] != 2) & (train_stances_cleaned['Stance'] != 3)]
train_stances_filtered

,Headline,Body ID,Stance
1,Hundreds of Palestinians flee floods in Gaza a...,158,0
4,Spider burrowed through tourist's stomach and ...,1923,1
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,0
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,0
11,Woman detained in Lebanon is not al-Baghdadi's...,1468,0
...,...,...,...
49887,Watch: US Marine saved by helmet after being s...,1734,0
49920,Tropical spider burrows under man's skin throu...,1883,0
49922,"Tour, agent: Tiger Woods not banned",757,0
49927,ESPN to save NFL's image with all-male domesti...,1500,1


In [190]:
merged_df = pd.merge(train_bodies, train_stances_filtered, on='Body ID', how='inner')

In [191]:
new_column_order = ['Body ID', 'Headline', 'articleBody', 'Stance']

merged_df = merged_df[new_column_order]

In [192]:
merged_df = merged_df.sort_values('Body ID', ascending = True)
display(merged_df.head())

,Body ID,Headline,articleBody,Stance
0,0,Small Meteorite Strikes in Nicaragua's Capital...,A small meteorite crashed into a wooded area i...,0
1,8,What was fake on the Internet this week: Obama...,There is so much fake stuff on the Internet in...,0
2,9,"Meteorite makes big crater in Nicaragua, gover...","(CNN) -- A meteorite crashed down in Managua, ...",0
3,9,Small Meteorite Strikes in Nicaragua's Capital...,"(CNN) -- A meteorite crashed down in Managua, ...",0
4,13,A soldier has been shot at Canada’s war memori...,A SOLDIER has been shot at Canada’s National W...,0


In [193]:
merged_df.to_csv('preprocessed_merged_data.csv', index=False)
print("\nPreprocessed and merged data saved to 'preprocessed_merged_data.csv'")


Preprocessed and merged data saved to 'preprocessed_merged_data.csv'
